# Some Demonstrations with Minikube and Containers

## Cleaning Up

First, the traces of former executions of  *minikube* are eliminated, the cluster itself (stop, delete) and further initialization steps in the cache directory to establish a clean base.

In [1]:
!minikube stop ; minikube delete ; rm -rf ~/.minikube/cache

✋  Stopping node "minikube"  ...
🛑  Powering off "minikube" via SSH ...
🛑  1 nodes stopped.
🔥  Deleting "minikube" in docker ...
🔥  Deleting container "minikube" ...
🔥  Removing /Users/lothar/.minikube/machines/minikube ...
💀  Removed all traces of the "minikube" cluster.


Second, formerly built podman images will be deleted as well.

In [2]:
!podman rmi $(podman images -qa) -f

Untagged: localhost/helloworld:v0.3.0
Untagged: localhost/helloworld:v0.2.0
Untagged: localhost/helloworld:v0.1.0
Untagged: docker.io/library/rust:1.55.0-alpine3.14
Untagged: docker.io/library/rust:1.55.0
Deleted: d4646741d2efe4c809d2114be35ca9e1993cbec40c7d3e00729563cb09f501db
Deleted: c2ac15af0b6e35c349d06610617ab1af497e098d983637f6b2c91c2cce8b891c
Deleted: bef5ddbb7b4c6feff5388ac452d12008633a9207e17d0931cd1719ee48c07acb
Deleted: cc0eb630c132ffeb122c19d2e5525c77bd3f814f6591e740af68142db5b4d119
Deleted: bcdc971133729fccfc2aea8075dbae6004e2c6756c87d8b5eb11f5ecce8e73f8
Deleted: 99e8ba4d29283f3c768470b2627ffde7ae50812a15b8de87a6842d4c65cc2685
Deleted: 46d71fed44c5c23fffcdc5af80fb621049e921e93e62792be3d18888a9726842
Deleted: a7c5d292027434284ca094543be814171a32f68410a52d3249db6be6a4def944
Deleted: decad7af277f79d62bf3371bcadb491cbcc00143cf8f744ce75523236dcb2f96
Deleted: 3ce9c7e20e3d50788aaf2972188e582c101533f9a39c6d8792300d9f2b77c629
Deleted: ac41586fb44e3aa14cee23647a0d7257d86f4a54963445

'Error: image name or ID must be specified' indicates that the cache has bees empty before.

In [3]:
!rm -rf alpine

In [4]:
!rm -rf helloworld

## Start The First Cluster

In [5]:
%%time

!KUBERNETES_VERSION=1.22.1 minikube start --kubernetes-version=${KUBERNETES_VERSION}

😄  minikube v1.23.0 on Darwin 11.5.2
✨  Automatically selected the docker driver. Other choices: hyperkit, virtualbox, ssh, podman (experimental)
👍  Starting control plane node minikube in cluster minikube
🚜  Pulling base image ...
💾  Downloading Kubernetes v1.22.1 preload ...
    > preloaded-images-k8s-v12-v1...: 515.04 MiB / 515.04 MiB  100.00% 5.91 MiB
🔥  Creating docker container (CPUs=2, Memory=1987MB) ...
🐳  Preparing Kubernetes v1.22.1 on Docker 20.10.8 ...
    ▪ Generating certificates and keys ...
    ▪ Booting up control plane ...
    ▪ Configuring RBAC rules ...
🔎  Verifying Kubernetes components...
    ▪ Using image gcr.io/k8s-minikube/storage-provisioner:v5
🌟  Enabled addons: storage-provisioner, default-storageclass
🏄  Done! kubectl is now configured to use "minikube" cluster and "default" namespace by default
CPU times: user 3.76 s, sys: 1.41 s, total: 5.18 s
Wall time: 2min 21s


## Checking Readiness

In [6]:
!kubectl get nodes

NAME       STATUS     ROLES                  AGE   VERSION
minikube   NotReady   control-plane,master   9s    v1.22.1


In [7]:
!sleep 10 ; kubectl get nodes

NAME       STATUS   ROLES                  AGE   VERSION
minikube   Ready    control-plane,master   19s   v1.22.1


## Start Cluster With Preloaded Cache

In [8]:
!minikube stop ; minikube delete

✋  Stopping node "minikube"  ...
🛑  Powering off "minikube" via SSH ...
🛑  1 nodes stopped.
🔥  Deleting "minikube" in docker ...
🔥  Deleting container "minikube" ...
🔥  Removing /Users/lothar/.minikube/machines/minikube ...
💀  Removed all traces of the "minikube" cluster.


In [9]:
%%time

!KUBERNETES_VERSION=1.22.1 minikube start --kubernetes-version=${KUBERNETES_VERSION}

😄  minikube v1.23.0 on Darwin 11.5.2
✨  Automatically selected the docker driver. Other choices: hyperkit, virtualbox, ssh, podman (experimental)
👍  Starting control plane node minikube in cluster minikube
🚜  Pulling base image ...
🔥  Creating docker container (CPUs=2, Memory=1987MB) ...
🐳  Preparing Kubernetes v1.22.1 on Docker 20.10.8 ...
    ▪ Generating certificates and keys ...
    ▪ Booting up control plane ...
    ▪ Configuring RBAC rules ...
🔎  Verifying Kubernetes components...
    ▪ Using image gcr.io/k8s-minikube/storage-provisioner:v5
🌟  Enabled addons: storage-provisioner, default-storageclass
🏄  Done! kubectl is now configured to use "minikube" cluster and "default" namespace by default
CPU times: user 842 ms, sys: 287 ms, total: 1.13 s
Wall time: 45.6 s


Having deleted the cluster the startuo time for a new cluster with prefilled caches reduces to well under a minute.

In [10]:
!sleep 10

## Creating First Namespace And Pod

In [11]:
%%bash

cat <<EOF| kubectl apply -f -
kind: Namespace
apiVersion: v1
metadata:
  name: demo
  labels:
    name: demo
---
kind: Pod
apiVersion: v1
metadata:
  name: demo-pod
  namespace: demo
spec:
  containers:
  - name: demo-ctr
    image: nginx:stable-alpine
    resources:
      limits:
        memory: "200Mi"
        cpu: "700m"
      requests:
        memory: "200Mi"
        cpu: "700m"
EOF

namespace/demo created
pod/demo-pod created


A namespace demo with a pod demo-pod in it according to above yaml.

In [12]:
%%bash

cat <<EOF| kubectl apply -f -
kind: Namespace
apiVersion: v1
metadata:
  name: demo
  labels:
    name: demo
---
kind: Pod
apiVersion: v1
metadata:
  name: demo-pod
  namespace: demo
spec:
  containers:
  - name: demo-ctr
    image: nginx:stable-alpine
    resources:
      limits:
        memory: "200Mi"
        cpu: "700m"
      requests:
        memory: "200Mi"
        cpu: "700m"
EOF

namespace/demo unchanged
pod/demo-pod unchanged


As nemspace *demo* and the pod *demo-pod* in it have already been created by executing the exact same command before, *kubectl apply* remains idempotent.

In [13]:
!kubectl get pods --namespace demo

NAME       READY   STATUS    RESTARTS   AGE
demo-pod   0/1     Pending   0          0s


The newly created and started pod *demo-pod* is just one pod among others constituting the base kubernetes cluster. Those containers are isolated in the *kube-system* namespace.

In [14]:
!kubectl get namespaces

NAME              STATUS   AGE
default           Active   18s
demo              Active   1s
kube-node-lease   Active   19s
kube-public       Active   20s
kube-system       Active   20s


In [15]:
!kubectl get pods --namespace kube-system

NAME                               READY   STATUS              RESTARTS   AGE
coredns-78fcd69978-hdkkp           0/1     ContainerCreating   0          3s
etcd-minikube                      1/1     Running             0          12s
kube-apiserver-minikube            1/1     Running             0          12s
kube-controller-manager-minikube   1/1     Running             0          12s
kube-proxy-hb48t                   0/1     ContainerCreating   0          3s
kube-scheduler-minikube            1/1     Running             0          12s
storage-provisioner                0/1     ContainerCreating   0          12s


In [16]:
%%bash

echo "systemd-cgls --no-pager ; exit" | minikube ssh

systemd-cgls --no-pager ; exit
]0;docker@minikube: ~docker@minikube:~$ systemd-cgls --no-pager ; exit
Control group /docker/bc31ea0d7864bce696283343b833410fcecb3e5f2b1db3a1aa517d1d82f7450c:
-.slice
├─init.scope 
│ └─1 /sbin/init
└─system.slice 
  ├─containerd.service 
  │ ├─ 199 /usr/bin/containerd
  │ ├─1601 /usr/bin/containerd-shim-runc-v2 -namespace moby -id 77afe6794e699b8…
  │ ├─1607 /usr/bin/containerd-shim-runc-v2 -namespace moby -id ac4c15bc18f9426…
  │ ├─1633 /usr/bin/containerd-shim-runc-v2 -namespace moby -id 3556eb9e120d8c1…
  │ ├─1672 /pause
  │ ├─1679 /pause
  │ ├─1690 /pause
  │ ├─1729 /usr/bin/containerd-shim-runc-v2 -namespace moby -id 9570f4abf799432…
  │ ├─1770 /pause
  │ ├─1800 /usr/bin/containerd-shim-runc-v2 -namespace moby -id a4932baea9274fc…
  │ ├─1822 /usr/bin/containerd-shim-runc-v2 -namespace moby -id 6782aec2cff5a3c…
  │ ├─1849 /usr/bin/containerd-shim-runc-v2 -namespace moby -id cea436f97b6e3fe…
  │ ├─1880 kube-scheduler --authentication-kubeconfig=/etc/

In [17]:
!skopeo --override-os linux copy docker://alpine:3.14.2 oci:alpine:3.14.2

Getting image source signatures
Copying blob a0d0a0d46f8b [=>--------------------------------] 199.5KiB / 2.7MiB
Copying blob a0d0a0d46f8b [===>------------------------------] 335.2KiB / 2.7MiB
Copying blob a0d0a0d46f8b [=====>----------------------------] 495.2KiB / 2.7MiB
Copying blob a0d0a0d46f8b [========>-------------------------] 751.2KiB / 2.7MiB
Copying blob a0d0a0d46f8b [=========>------------------------] 799.2KiB / 2.7MiB
Copying blob a0d0a0d46f8b [====================>---------------] 1.6MiB / 2.7MiB
Copying blob a0d0a0d46f8b [=================================>--] 2.6MiB / 2.7MiB
Copying blob a0d0a0d46f8b [=================================>--] 2.6MiB / 2.7MiB
Copying blob a0d0a0d46f8b [====================================] 2.7MiB / 2.7MiB
Copying blob a0d0a0d46f8b done  
Copying blob a0d0a0d46f8b done  
Copying blob a0d0a0d46f8b done  
Copying config 696d33ca15 [====================================] 585.0b / 585.0b
Copying config 696d33ca15 done  
Copying config 696d33ca15 

In [18]:
!tree alpine

alpine
├── blobs
│   └── sha256
│       ├── 03014f0323753134bf6399ffbe26dcd75e89c6a7429adfab392d64706649f07b
│       ├── 696d33ca1510966c426bdcc0daf05f75990d68c4eb820f615edccf7b971935e7
│       └── a0d0a0d46f8b52473982a3c466318f479767577551a53ffc9074c9fa7035982e
├── index.json
└── oci-layout

2 directories, 5 files


In [19]:
!file alpine/blobs/sha256/*

alpine/blobs/sha256/03014f0323753134bf6399ffbe26dcd75e89c6a7429adfab392d64706649f07b: JSON data
alpine/blobs/sha256/696d33ca1510966c426bdcc0daf05f75990d68c4eb820f615edccf7b971935e7: JSON data
alpine/blobs/sha256/a0d0a0d46f8b52473982a3c466318f479767577551a53ffc9074c9fa7035982e: gzip compressed data, original size modulo 2^32 5865472


In [20]:
!cat alpine/blobs/sha256/03014f0323753134bf6399ffbe26dcd75e89c6a7429adfab392d64706649f07b | jq

{
  "schemaVersion": 2,
  "config": {
    "mediaType": "application/vnd.oci.image.config.v1+json",
    "digest": "sha256:696d33ca1510966c426bdcc0daf05f75990d68c4eb820f615edccf7b971935e7",
    "size": 585
  },
  "layers": [
    {
      "mediaType": "application/vnd.oci.image.layer.v1.tar+gzip",
      "digest": "sha256:a0d0a0d46f8b52473982a3c466318f479767577551a53ffc9074c9fa7035982e",
      "size": 2814446
    }
  ]
}


In [21]:
!cat alpine/blobs/sha256/696d33ca1510966c426bdcc0daf05f75990d68c4eb820f615edccf7b971935e7 | jq

{
  "created": "2021-08-27T17:19:45.758611523Z",
  "architecture": "amd64",
  "os": "linux",
  "config": {
    "Env": [
      "PATH=/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin"
    ],
    "Cmd": [
      "/bin/sh"
    ]
  },
  "rootfs": {
    "type": "layers",
    "diff_ids": [
      "sha256:e2eb06d8af8218cfec8210147357a68b7e13f7c485b991c288c2d01dc228bb68"
    ]
  },
  "history": [
    {
      "created": "2021-08-27T17:19:45.553092363Z",
      "created_by": "/bin/sh -c #(nop) ADD file:aad4290d27580cc1a094ffaf98c3ca2fc5d699fe695dfb8e6e9fac20f1129450 in / "
    },
    {
      "created": "2021-08-27T17:19:45.758611523Z",
      "created_by": "/bin/sh -c #(nop)  CMD [\"/bin/sh\"]",
      "empty_layer": true
    }
  ]
}


In [22]:
!rustup self uninstall -y

info: removing rustup home
info: removing cargo home
info: removing rustup binaries
info: rustup is uninstalled


In [23]:
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y

info: downloading installer
info: profile set to 'default'
info: default host triple is x86_64-apple-darwin
info: syncing channel updates for 'stable-x86_64-apple-darwin'
info: latest update on 2021-09-09, rust version 1.55.0 (c8dfcfe04 2021-09-06)
info: downloading component 'cargo'
  4.4 MiB /   4.4 MiB (100 %)   1.4 MiB/s in  3s ETA:  0s
info: downloading component 'clippy'
  1.7 MiB /   1.7 MiB (100 %)   1.2 MiB/s in  1s ETA:  0s
info: downloading component 'rust-docs'
 17.1 MiB /  17.1 MiB (100 %)   3.9 MiB/s in  4s ETA:  0s
info: downloading component 'rust-std'
 21.0 MiB /  21.0 MiB (100 %)   5.2 MiB/s in  4s ETA:  0s
info: downloading component 'rustc'
 75.9 MiB /  75.9 MiB (100 %)   4.1 MiB/s in 17s ETA:  0s
info: downloading component 'rustfmt'
info: installing component 'cargo'
info: installing component 'clippy'
info: installing component 'rust-docs'
 17.1 MiB /  17.1 MiB (100 %)   5.0 MiB/s in  2s ETA:  0s
info: installing component 'rust-std'
 21.0 MiB /  21.0 MiB (100 %)

In [24]:
!rustup default stable

info: using existing install for 'stable-x86_64-apple-darwin'
info: default toolchain set to 'stable-x86_64-apple-darwin'

  stable-x86_64-apple-darwin unchanged - rustc 1.55.0 (c8dfcfe04 2021-09-06)



In [25]:
!rustup update

info: syncing channel updates for 'stable-x86_64-apple-darwin'
info: checking for self-updates

  stable-x86_64-apple-darwin unchanged - rustc 1.55.0 (c8dfcfe04 2021-09-06)

info: cleaning up downloads & tmp directories


In [26]:
!cargo new helloworld

     Created binary (application) `helloworld` package


In [27]:
%%bash

cd helloworld

cat >Dockerfile <<EOF
FROM docker.io/library/rust:1.55.0

WORKDIR /usr/src/helloworld

COPY . .

RUN cargo build --release

RUN cargo install --path .

CMD ["/usr/local/cargo/bin/helloworld"]
EOF

In [28]:
%%bash

cd helloworld

cat >Dockerfile.slim <<EOF
FROM docker.io/library/rust:1.55.0-alpine3.14

WORKDIR /usr/src/helloworld

COPY . .

RUN cargo build --release

RUN cargo install --path .

CMD ["/usr/local/cargo/bin/helloworld"]
EOF

In [29]:
%%bash

cd helloworld

cat >Dockerfile.slim-staged <<EOF
FROM docker.io/library/rust:1.55.0-alpine3.14

WORKDIR /usr/src/helloworld

COPY . .

RUN cargo build --release

RUN cargo install --path .

CMD ["/usr/local/cargo/bin/helloworld"]
EOF

In [30]:
!podman info

host:
  arch: amd64
  buildahVersion: 1.22.3
  cgroupControllers: []
  cgroupManager: systemd
  cgroupVersion: v2
  conmon:
    package: conmon-2.0.29-2.fc34.x86_64
    path: /usr/bin/conmon
    version: 'conmon version 2.0.29, commit: '
  cpus: 1
  distribution:
    distribution: fedora
    version: "34"
  eventLogger: journald
  hostname: localhost
  idMappings:
    gidmap:
    - container_id: 0
      host_id: 1000
      size: 1
    - container_id: 1
      host_id: 100000
      size: 65536
    uidmap:
    - container_id: 0
      host_id: 1000
      size: 1
    - container_id: 1
      host_id: 100000
      size: 65536
  kernel: 5.13.13-200.fc34.x86_64
  linkmode: dynamic
  memFree: 1711087616
  memTotal: 2061852672
  ociRuntime:
    name: crun
    package: crun-1.0-1.fc34.x86_64
    path: /usr/bin/crun
    version: |-
      crun version 1.0
      commit: 139dc6971e2f1d931af520188763e984d6cdfbf8
      spec: 1.0.0
      +SYSTEMD +SELINUX +APPARMOR +CAP +SECCOMP +EBPF +CRIU +YAJL
  os: l

In [31]:
!cd helloworld ; podman build --tag helloworld:v0.1.0 --file Dockerfile .

STEP 1/6: FROM docker.io/library/rust:1.55.0
Trying to pull docker.io/library/rust:1.55.0...
Getting image source signatures
Copying blob sha256:74ca45178655a5c9b4a372bb131e0e6a563074c81321f9178fd06bc9847f6559
Copying blob sha256:955615a668ce169f8a1443fc6b6e6215f43fe0babfb4790712a2d3171f34d366
Copying blob sha256:2756ef5f69a5190f4308619e0f446d95f5515eef4a814dbad0bcebbbbc7b25a8
Copying blob sha256:911ea9f2bd51e53a455297e0631e18a72a86d7e2c8e1807176e80f991bde5d64
Copying blob sha256:27b0a22ee906271a6ce9ddd1754fdd7d3b59078e0b57b6cc054c7ed7ac301587
Copying blob sha256:8584d51a9262f9a3a436dea09ba40fa50f85802018f9bd299eee1bf538481077
Copying blob sha256:8584d51a9262f9a3a436dea09ba40fa50f85802018f9bd299eee1bf538481077
Copying blob sha256:955615a668ce169f8a1443fc6b6e6215f43fe0babfb4790712a2d3171f34d366
Copying blob sha256:27b0a22ee906271a6ce9ddd1754fdd7d3b59078e0b57b6cc054c7ed7ac301587
Copying blob sha256:74ca45178655a5c9b4a372bb131e0e6a563074c81321f9178fd06bc9847f6559
Copying blob sha256:2756e

In [32]:
!podman images

REPOSITORY              TAG         IMAGE ID      CREATED       SIZE
localhost/helloworld    v0.1.0      e2881839f7ba  25 hours ago  1.28 GB
docker.io/library/rust  1.55.0      378164a392dd  3 days ago    1.27 GB


In [33]:
!podman run --name helloworld --rm helloworld:v0.1.0

Hello, world!


In [34]:
!cd helloworld ; podman build --tag helloworld:v0.2.0 --file Dockerfile.slim .

STEP 1/6: FROM docker.io/library/rust:1.55.0-alpine3.14
Trying to pull docker.io/library/rust:1.55.0-alpine3.14...
Getting image source signatures
Copying blob sha256:922454686f1ed186a1f1ecf76b947b3b16125d09227ba5d5736ce77c72941ce0
Copying blob sha256:a0d0a0d46f8b52473982a3c466318f479767577551a53ffc9074c9fa7035982e
Copying blob sha256:d4ea2bb3df1724da0bb9bbf8d43f7a5b7192610627c590c2fd4b8e0770230cbf
Copying blob sha256:a0d0a0d46f8b52473982a3c466318f479767577551a53ffc9074c9fa7035982e
Copying blob sha256:d4ea2bb3df1724da0bb9bbf8d43f7a5b7192610627c590c2fd4b8e0770230cbf
Copying blob sha256:922454686f1ed186a1f1ecf76b947b3b16125d09227ba5d5736ce77c72941ce0
Copying config sha256:ac41586fb44e3aa14cee23647a0d7257d86f4a549634453ab95497bd24eff193
Writing manifest to image destination
Storing signatures
STEP 2/6: WORKDIR /usr/src/helloworld
--> ae06f670ae0
STEP 3/6: COPY . .
--> 6687bea1707
STEP 4/6: RUN cargo build --release
   Compiling helloworld v0.1.0 (/usr/src/helloworld)
    Finished release 

In [35]:
!cd helloworld ; podman build --tag helloworld:v0.3.0 --file Dockerfile.slim-staged .

STEP 1/6: FROM docker.io/library/rust:1.55.0-alpine3.14
STEP 2/6: WORKDIR /usr/src/helloworld
--> Using cache ae06f670ae029987c0ef4469f8a0eebec6d81fe7e8016e27efa3b6a24c990a2b
--> ae06f670ae0
STEP 3/6: COPY . .
--> Using cache 6687bea1707ed4bcd2fec606682cd6ef6f1f2196ef10a48daa17f1760d363da0
--> 6687bea1707
STEP 4/6: RUN cargo build --release
--> Using cache 787ea48a7ec0f609fff798ab000487ab6cb0a415a4e1d2de387b8cb5616d4c0e
--> 787ea48a7ec
STEP 5/6: RUN cargo install --path .
--> Using cache 317fc798556fc712a6b8fedf2d3be704939c14e9e50982d93bbe01688b0cedf5
--> 317fc798556
STEP 6/6: CMD ["/usr/local/cargo/bin/helloworld"]
--> Using cache bc980322c00e80ed1ca96bb24590c9bf275db89982ddaac1148096d090c3c340
COMMIT helloworld:v0.3.0
--> bc980322c00
Successfully tagged localhost/helloworld:v0.3.0
Successfully tagged localhost/helloworld:v0.2.0
bc980322c00e80ed1ca96bb24590c9bf275db89982ddaac1148096d090c3c340


In [36]:
!podman images

REPOSITORY              TAG                IMAGE ID      CREATED       SIZE
localhost/helloworld    v0.3.0             bc980322c00e  25 hours ago  880 MB
localhost/helloworld    v0.2.0             bc980322c00e  25 hours ago  880 MB
localhost/helloworld    v0.1.0             e2881839f7ba  25 hours ago  1.28 GB
docker.io/library/rust  1.55.0-alpine3.14  ac41586fb44e  3 days ago    868 MB
docker.io/library/rust  1.55.0             378164a392dd  3 days ago    1.27 GB


In [37]:
!podman run --name helloworld --rm helloworld:v0.3.0

Hello, world!
